In [14]:
%%capture
!pip install wandb --upgrade

# workaround to fetch MNIST data
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

In [20]:
import wandb
import pprint

wandb.login()

True

In [21]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric
pprint.pprint(sweep_config)

{'method': 'random', 'metric': {'goal': 'minimize', 'name': 'loss'}}


In [22]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

pprint.pprint(parameters_dict)

{'dropout': {'values': [0.3, 0.4, 0.5]},
 'fc_layer_size': {'values': [128, 256, 512]},
 'optimizer': {'values': ['adam', 'sgd']}}


In [23]:
# Set value directly for hyperparams that you don't want to change

parameters_dict.update({
    'epochs': {
        'value': 1}
    })
pprint.pprint(parameters_dict)

{'dropout': {'values': [0.3, 0.4, 0.5]},
 'epochs': {'value': 1},
 'fc_layer_size': {'values': [128, 256, 512]},
 'optimizer': {'values': ['adam', 'sgd']}}


In [24]:
import math

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(32),
        'max': math.log(256),
      }
    })
pprint.pprint(parameters_dict)

{'batch_size': {'distribution': 'q_log_uniform',
                'max': 5.545177444479562,
                'min': 3.4657359027997265,
                'q': 1},
 'dropout': {'values': [0.3, 0.4, 0.5]},
 'epochs': {'value': 1},
 'fc_layer_size': {'values': [128, 256, 512]},
 'learning_rate': {'distribution': 'uniform', 'max': 0.1, 'min': 0},
 'optimizer': {'values': ['adam', 'sgd']}}


In [25]:
# View all the sweep configurations
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform',
                               'max': 5.545177444479562,
                               'min': 3.4657359027997265,
                               'q': 1},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [26]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: 95kk1tcx
Sweep URL: https://wandb.ai/egealtan/pytorch-sweeps-demo/sweeps/95kk1tcx


In [27]:
# Define training procedure:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})

def build_dataset(batch_size):
   
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    # download MNIST training dataset
    dataset = datasets.MNIST(".", train=True, download=True,
                             transform=transform)
    sub_dataset = torch.utils.data.Subset(
        dataset, indices=range(0, len(dataset), 5))
    loader = torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size)

    return loader


def build_network(fc_layer_size, dropout):
    network = nn.Sequential(  # fully-connected, single hidden layer
        nn.Flatten(),
        nn.Linear(784, fc_layer_size), nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(fc_layer_size, 10),
        nn.LogSoftmax(dim=1))

    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.nll_loss(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader) 

In [30]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: lt4f4pxs with config:
wandb: 	batch_size: 153
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.06234394024540954
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


batch loss,█▇▅▄▃▃▂▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▁▂▂▁▁▁▁▁▁▂
epoch,▁
loss,▁
batch loss,0.50939
epoch,0
loss,0.57624


wandb: Agent Starting Run: 3rs8r7hy with config:
wandb: 	batch_size: 92
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.04565907666911791
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


batch loss,██▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,0.59535
epoch,0
loss,1.62059


wandb: Agent Starting Run: mmvow3p0 with config:
wandb: 	batch_size: 112
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.016384689226663886
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


batch loss,██▇▆▅▅▃▃▃▃▂▃▂▂▃▃▃▂▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁
epoch,▁
loss,▁
batch loss,0.10435
epoch,0
loss,0.66731


wandb: Agent Starting Run: j3ccge8x with config:
wandb: 	batch_size: 120
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.04174113392626603
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


batch loss,██▆▅▃▃▂▂▂▂▂▂▂▃▂▃▃▂▂▂▂▂▂▁▁▂▁▂▁▂▁▂▁▂▁▂▁▁▁▁
epoch,▁
loss,▁
batch loss,0.2972
epoch,0
loss,0.66232


wandb: Agent Starting Run: bxruvtno with config:
wandb: 	batch_size: 55
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.02219320305539827
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


batch loss,█▇▅▃▂▄▃▂▂▂▁▂▂▂▃▂▂▂▂▁▃▄▁▁▁▂▂▂▁▁▁▂▂▂▂▂▂▁▁▂
epoch,▁
loss,▁
batch loss,0.20847
epoch,0
loss,0.59192


In [ ]:
#Modular Layers

# class MLP(nn.Module):
#     def __init__(self, h_sizes, out_size):
#         ...
#         self.hidden = nn.ModuleList()
#         for k in range(len(h_sizes)-1):
#             self.hidden.append(nn.Linear(h_sizes[k], h_sizes[k+1]))
#         ...